<a href="https://colab.research.google.com/github/gunterno/PCDS/blob/main/P7A_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

___Inspiré par le Kernel :___



---

Société financière qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.


L’entreprise souhaite **développer un modèle de scoring de la probabilité de défaut de paiement du client** pour étayer la décision d'accorder ou non un prêt à un client potentiel en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de **transparence** vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

Elle décide donc de développer un **dashboard interactif** pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement. 

### Mission

Présenter son travail de modélisation à l'oral


- Réaliser un dashboard pour présenter son travail de modélisation
-Rédiger une note méthodologique afin de communiquer sa démarche de modélisation
-Utiliser un logiciel de version de code pour assurer l’intégration du modèle
- Déployer un modèle via une API dans le Web

Michaël, manager, incite à sélectionner un kernel Kaggle pour faciliter la préparation des données nécessaires à l’élaboration du modèle de scoring. L'idée est d'analyser ce kernel pour l'adapter aux besoins de la mission.

### Spécifications du dashboard
Michaël fourni un cahier des charges pour le dashboard interactif. Celui-ci devra a minima contenir les fonctionnalités suivantes :

- Permettre de visualiser le score et l’interprétation de ce score pour chaque client de façon intelligible pour une personne non experte en data science.
- Permettre de visualiser des informations descriptives relatives à un client (via un système de filtre).
- Permettre de comparer les informations descriptives relatives à un client à l’ensemble des clients ou à un groupe de clients similaires.

---

In [ ]:
#Essential data science libraries
import pandas as pd
import seaborn as sns
import glob
import pickle
import os
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [ ]:
#Graphing…
plt.style.use('fivethirtyeight')
plt.rcParams.update(
    {
     'xtick.labelsize':25,
     'ytick.labelsize':25,
     'axes.labelsize': 25,
     'legend.fontsize': 25,
     'axes.titlesize':45,
     'axes.titleweight':'bold',
     'axes.titleweight':'bold'
    })

## Chargement des données et Analyse exploratoire

### Data description from [Kaggle](https://www.kaggle.com/c/home-credit-default-risk/data) : 


#### application_{train|test}.csv
This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data sample.

#### bureau.csv
All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

#### bureau_balance.csv
Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

#### POS_CASH_balance.csv
Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

#### credit_card_balance.csv
Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

#### previous_application.csv
All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

#### installments_payments.csv
Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

#### HomeCredit_columns_description.csv
This file contains descriptions for the columns in the various data files.

In [ ]:

## Fonctions
 
###Fonction valmanque, paramètre (dataframe, pourcent,1 ou 0)
###Imprime les colonnes avec le nombre de ligne avec valeurs ainsi que le pourcentage sans valeurs
###puis retourne une liste contenant les nom de colonnes ayant un pourcentage inférieur a la valeur transmise.
### retourne aussi le nombre de lignes non nulles correspondant à la colonne ValNa
 
def valmanque(df,perct,imp):
  # recherche des valeurs manquantes
  NutriColonne=df.columns.values
  ListeNaN=[]
  ValNaN=df.isna().sum()
  #nb Lignes
  LigneNonNull=[]
  NbLignes=len(df)
  if imp == 1:
    print("nombre de lignes :",NbLignes)
    print("nombre de ligne contenant des valeurs nul:")
    print(ValNaN)
    print()
    print("données par ligne après supression des valeurs null")
 
  i=0
  i2=0
  while i< len(df.columns):
    LigneNonNull.append(NbLignes-ValNaN[i])
 
    if imp == 1 and (ValNaN[i]*100/NbLignes)>=perct:
        print('-----------------------------------------')
        print(NutriColonne[i])
        print(NbLignes-ValNaN[i],' nombres de lignes remplies')
        print(ValNaN[i]*100/NbLignes,' % vide')
    
    if (ValNaN[i]*100/NbLignes)>=perct:
      ListeNaN.append(NutriColonne[i])
      i2=i2+1
    i=i+1
  print(i2,'sur',i,'variable avec plus de ',perct, 'pourcent de valeurs manquantes')
  return(ListeNaN)
 
#fonction donnant des données staistiques de bases en indiquant un dataframe et une liste de  noms de colonnes
#retourne graphiques de corrélations et un tableau de résultat
 
def Analyse_1(df,Var):  
  # Analyse statistique de base (describe fait pareil,mais je voulais ma version...)
  #création du dataframe
  Resume = pd.DataFrame(columns=['minimum','maximum','somme','moyenne','médiane','mode,','varianceVar','varianceStd'])
 
  for bcl in Var:
   
   mini=(df[bcl].min())
   
   maxi=(df[bcl].max())
 
   total=(df[bcl].sum())
  
   moy=(df[bcl].mean())
   
   mediane=(df[bcl].median())
  
   Modee=(df[bcl].mode())
   
   Varr=(df[bcl].var(ddof=0))
  
   Stdd=(df[bcl].std(ddof=0))
   
   #création du dataframe de résultat  'ajout row'
   df_new_row = pd.DataFrame(data=np.array([[mini,maxi,total,moy,mediane,Modee,Varr,Stdd]]), columns=['minimum','maximum','somme','moyenne','médiane','mode,','varianceVar','varianceStd'])
   Resume = pd.concat([Resume,df_new_row], ignore_index=True)
 
   plt.plot(df[bcl])
   plt.show()
  insert_index = 0
  insert_colname = 'Nom'
  insert_values = Var # this can be a numpy array too
  Resume.insert(loc=insert_index, column=insert_colname, value=insert_values)
  return(Resume)
 
# Fonction créant un graphique de corellation en indiquant du dataframe et le nom de l'indicateur
def correl(df,indicateur):
 
  corr=df.corr(method=indicateur)
  plt.figure(figsize = (16,5))
 
  sns.heatmap(corr, 
            xticklabels=corr.columns,
            yticklabels=corr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=1)
  
  def plot_stat(data, feature, title) : 
    
    ax, fig = plt.subplots(figsize=(20,8)) 
    ax = sns.countplot(y=feature, data=data, order=data[feature].value_counts(ascending=False).index)
    ax.set_title(title)

    for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_width()/len(data[feature]))
                x = p.get_x() + p.get_width()
                y = p.get_y() + p.get_height()/2
                ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')

    show()

    def plot_percent_target1(data, feature, title) : 
    
    cat_perc = data[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='TARGET', ascending=False, inplace=True)
    
    ax, fig = plt.subplots(figsize=(20,8)) 
    ax = sns.barplot(y=feature, x='TARGET', data=cat_perc)
    ax.set_title(title)
    ax.set_xlabel("")
    ax.set_ylabel("Percent of target with value 1")

    for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_width())
                x = p.get_x() + p.get_width()
                y = p.get_y() + p.get_height()/2
                ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')

    show()

IndentationError: ignored

In [ ]:
print("CHECKLIST DATA EXPLORER :")


In [ ]:
from google.colab import drive

drive.mount("/content/drive")



application_train = pd.read_csv("/content/drive/MyDrive/P7/data/application_train.csv")
application_test = pd.read_csv("/content/drive/MyDrive/P7/data/application_test.csv")
bureau = pd.read_csv("/content/drive/MyDrive/P7/data/bureau.csv")
#bureau_balance = pd.read_csv("/content/drive/MyDrive/P7/data/bureau_balance.csv")
#credit_card_balance = pd.read_csv("/content/drive/MyDrive/P7/data/credit_card_balance.csv")
installments_payments = pd.read_csv("/content/drive/MyDrive/P7/data/installments_payments.csv")
POS_CASH_balance = pd.read_csv("/content/drive/MyDrive/P7/data/POS_CASH_balance.csv")
previous_application = pd.read_csv("/content/drive/MyDrive/P7/data/previous_application.csv")
description = pd.read_csv("/content/drive/MyDrive/P7/data/HomeCredit_columns_description.csv",encoding="ISO-8859-1")

In [ ]:
print("Explications des tables et colonnes :")
pd.set_option("max_colwidth", 400)

description

### EDA application_train.csv | test.csv


In [ ]:
train=application_train.copy()
test=application_test.copy()

In [ ]:
print('taille de : train ', train.shape)
print('taille de : test ', test.shape)

la colonne manquante dans target et la colonne target

In [ ]:
#Distribution de la colonne'TARGET' entre les 1 crédit non remboursés et 0 crédit remboursés
application_train['TARGET'].value_counts()

In [ ]:
ax, fig = plt.subplots(figsize=(20,8)) 
ax = sns.countplot(y='TARGET', data=train)
ax.set_title("distribution des crédits accetpés et refusé")

for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/len(application_train.TARGET))
        x = p.get_x() + p.get_width()
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')
        
show()

La pluspart des prêts sont remboursés à temps. Une fois que nous entrons dans des modèles d'apprentissage automatique plus sophistiqués, nous pouvons pondérer les classes par leur représentation dans les données pour refléter ce déséquilibre.

### Types de colonnes
Analyse rapide du type de features, mais surtout établir un aperçu afin de pouvoir poser une réflexion sur l'encodage des données catégorielles.

In [ ]:
#Number of each type of column
application_train.dtypes.value_counts()




#


In [ ]:

#Create a simple dataset with the train / test merge app
#supression de la collonne taget pour avoir une liste sans la réponse

liste_clients = train.append(test)
del(train['TARGET'])

print('Train:' + str(train.shape))
print('Test:' + str(test.shape))
print('liste_clients:' + str(liste_clients.shape))


KeyError: ignored

ANALISE

In [ ]:
liste_clients.describe()

#57 sur 121 variable avec plus de  10 pourcent de valeurs manquantes


In [ ]:
#donction perso pour valeur manquantes
valmanque(liste_clients,10,0)

In [ ]:
#donction perso pour valeur manquantes
valmanque(liste_clients,50,0)

"37 sur 121 variable avec plus de  50 pourcent de valeurs manquantes


# Features engineering
Nous pourrais enrichir et retravailler, ou créer de nouvelle variables, mais ceci n'étant pas le centre du projet je me contenterai de suprimmer quelques colonnes que je trouve mal adaptées ou inapropriées.


La plupart des variables catégorielles ont un nombre relativement petit d'entrées uniques. Nous devrons trouver un moyen de traiter ces variables catégorielles…

[texte du lien](https://)### Valeurs manquantes
En modélisation, des modèles tels que XGBoost peuvent gérer les valeurs manquantes sans imputation. Plusieurs alternatives seront possibles: remplacer les NaN, supprimer les colonnes avec un pourcentage élevé de valeurs manquantes (impossible de savoir à l'avance si ces colonnes seront utiles à notre modèle). Dans l'immédiat toutes les colonnes sont conservées…

In [ ]:
#Global view of the missing values (black)
plt.figure(figsize=(20,10))
sns.heatmap(train.notna(), cbar=False)
show()

Une synthèse des données manquantes un peu confuse du fait du grand nombre de variables, mais il se dégage un premier constat qui montre que les NaN sont plus fortement présentent sur les caractéristiques des habitats (et non sur les crédits), comme l'atteste le TOP 10 ci-après…

In [ ]:
def nan_check(data):
    '''Check Missing Values'''
    total = data.isnull().sum()
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 2))
    missing_data = pd.concat([total, percent_2], 
                             axis=1, keys=['Total', '%']).sort_values('%', ascending=False)
    return missing_data

print('TOP 20 Missing values from Training dataset')
nan_check(liste_clients)[:20]

LightGBM utilise NA (NaN) pour représenter les valeurs manquantes par défaut. 
Nous allons garder cette option.

#Outliers

In [ ]:
#How many days before the application the perso... (def. from HomeCredit_columns_description.csv)
liste_clients['DAYS_EMPLOYED'].describe()

In [ ]:
print("%0.0f valeur 1000 ans 'days employed' " % 
      len(liste_clients[liste_clients['DAYS_EMPLOYED'] == 365243]))


In [ ]:
365243 est la valeur maximale pour DAYS_EMPLOYED elle représente 1000 ans, il est curieux de retrouver ce chifres de très nombreuse fois
celà ressemble à une valeur impossible placée là pour le programme de la banque.

#Les seuls outliers semblent voulu

### analyse 


**Loan types -** Distribution du type de prêts contractés + comparatif avec le pourcentage des prêts avec la valeur TARGET 1(prêt non retourné).

Les prêts renouvelables ne représentent qu'une petite fraction (10%) du nombre total de prêts; dans le même temps, un plus grand nombre de crédits renouvelables, par rapport à leur fréquence, ne sont pas remboursés.

**Client gender -** Bien que deux fois plus de prêt rembourser vienne de femmes cel me semble un facteur trop discriminant pour l'éthique de la banque colonne supprimée

In [ ]:
#
plot_stat(liste_clients, 'CODE_GENDER',"sex distribution total")
print("                                   -------------------------------------------------------")
plot_percent_target1(liste_clients, 'CODE_GENDER',"non emboursement entre les diférens sex")

Le nombre de clients féminins est presque le double du nombre de clients masculins. En ce qui concerne le pourcentage de crédits en souffrance, les hommes ont plus de chances de ne pas rembourser leurs prêts (10%), comparativement aux femmes (7%).

XNA dot être prévue pour quelque chose qui n'a pas été utilisé

In [ ]:
del(liste_clients['CODE_GENDER'])


La pocession d'un véhicule ou non me semble non adéquate
regardons

**Flag own car -** Distribution d'un impact possible entre les clients propriétaire d'un véhicule et ceux qui ne le sont pas…

In [ ]:
#FLAG_OWN_CAR
plot_stat(application_train, 'FLAG_OWN_CAR',"Personnes possédant une voiture")
print("                                   -------------------------------------------------------")
plot_percent_target1(application_train, 'FLAG_OWN_CAR',"Non remboursement")

#Comme j'y avais pensé celà n'est pas représentatif.

In [ ]:
del(liste_clients['FLAG_OWN_CAR'])

## Features engineering
On pourrais enrichir et retravailler, ou créer de nouvelle variables, mais ceci n'étant pas le centre du projet je me contenterai de suprimmer quelques colonnes que je trouve mal adaptées ou inapropriées.

Les deux jeux de données ont exactement le même format avec une seule différence, la TARGET dispo dans le train.

# le dataframe liste_clients est utilisé comme dataframe complet

Mais nous utiliserons train pour l'entrainement et test pour le test puisque nous n'avons pas les données complêtes (test) ne contient pas le résultat.

## Preprocessing des données

### Split train / test data
Il est nécessaire de commencer par la mise en place des données d'entrainement / test. On peut procéder en rappel avec les jeux de données application_train/test.

### Encoding categorical features 


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Create a label encoder object
le = LabelEncoder()
count = 0

# Iterate through the columns
for col in data_train:
    # If 2 or fewer unique categories
    if data_train.loc[:,col].dtype == 'object' and len(list(data_train.loc[:,col].unique())) <= 2:
        # Train on the training data
        le.fit(data_train.loc[:,col])
        # Transform both training and testing data
        data_train.loc[:,col] = le.transform(data_train.loc[:,col])
        data_test.loc[:,col] = le.transform(data_test.loc[:,col])

        count += 1
            
print('%d columns were label encoded.' % count)

In [ ]:
print('Training Features shape with categorical columns: ', data_train.shape)
print('Testing Features shape with categorical columns: ', data_test.shape)